# Costumer Relationship Management (CRM)

No cenário empresarial atual, caracterizado pela rapidez e alta competitividade, o sucesso de qualquer organização vai além dos produtos ou serviços que oferece.\
Ele também depende fortemente da habilidade de estabelecer e manter relações sólidas e duradouras com os clientes. É aqui que entra a **Gestão de Relacionamento com o Cliente (CRM)**. \
A **CRM** é uma estratégia essencial que as empresas utilizam para gerenciar interações e relacionamentos com seus clientes, visando maximizar a *satisfação*, a *fidelidade* e a *rentabilidade* desses clientes.

A CRM envolve a coleta e análise de dados sobre interações, preferências e comportamentos dos clientes, permitindo a adaptação de produtos, serviços e estratégias de marketing. Com os sistemas de CRM, as empresas podem entender melhor seus clientes, antecipar suas necessidades e oferecer a mensagem ou solução adequada no momento certo.

In [1]:
import pandas as pd
import datetime as dt
import csv

EDA 

In [2]:
df = pd.read_excel('online_retail_2009_2010.xlsx')

In [3]:
df_copy = df.copy()

In [4]:
# Shape
print(df_copy.shape)

(525461, 8)


In [5]:
# Number of NaNs
print(df_copy.isnull().sum())

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [6]:
# We will drop the rows that contain NaN values in Customer ID or Description 
# columns.
df_copy.dropna(inplace=True)


In [7]:
# Number of unique products 
print(df_copy["Description"].nunique())

4459


In [8]:
# Number of customers purchasing each produc
df_copy['Description'].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3245
REGENCY CAKESTAND 3 TIER              1872
STRAWBERRY CERAMIC TRINKET BOX        1536
ASSORTED COLOUR BIRD ORNAMENT         1376
HOME BUILDING BLOCK WORD              1229
                                      ... 
BLUE OWL DECORATION                      1
PINK OWL DECORATION                      1
IVORY WALL CLOCK                         1
RED WALL CLOCK                           1
BAKING MOULD EASTER EGG MILK CHOC        1
Name: count, Length: 4459, dtype: int64

In [9]:
# Let's see how many units of each product were purchased.
df_copy.groupby("Description").agg({"Quantity": "sum"}).sort_values(by = 'Quantity', ascending=True)

,Quantity
Description,
Discount,-1675
WHITE CHERRY LIGHTS,-97
SILVER CHERRY LIGHTS,-96
LARGE RED RETROSPOT WINDMILL,-64
TREE OF NOAH FESTIVE SCENTED CANDLE,-34
...,...
ASSORTED COLOUR BIRD ORNAMENT,44120
PACK OF 72 RETRO SPOT CAKE CASES,44507
BROCADE RING PURSE,47430


In [10]:
# However, we see that the Quantity variable 
# is sometimes negative. There seems to be a problem here.
# The reason for this is that canceled transactions are entered 
# into the system as negative. We will fix it soon.

In [11]:
# Above, we observed that the product named WHITE HANGING HEART T-LIGHT HOLDER 
# was purchased by 3549 different customers. Below, we see that 57733 units 
# of the product named WHITE HANGING HEART T-LIGHT HOLDER were sold.

In [12]:
# A total of 28816 different invoices were issued.
df_copy["Invoice"].nunique() # 28816


23587

In [13]:
# The cost of each invoice
df_copy["TotalPrice"] = df_copy["Quantity"] * df_copy["Price"]
df_copy.groupby("Invoice").agg({"TotalPrice": "sum"})

,TotalPrice
Invoice,
489434,505.30
489435,145.80
489436,630.33
489437,310.75
489438,2286.24
...,...
C538121,-12.75
C538122,-1.25
C538123,-7.50


In [14]:
# Some values in Invoice column start with 'C'. It means cancelled transaction. 
# For example:
df_copy[df_copy["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.4
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia,-9.9
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia,-17.0
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia,-12.6
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia,-35.4


In [15]:
# We don't care cancelled transaction here.. So we will just ignore them, 
# and apply RFM analysis.
df_copy = df_copy[~df_copy["Invoice"].str.contains("C", na=False)]

In [16]:
df_copy.to_csv('2009_2010.csv', index = False)

CALCULANDO AS MÉTRICAS

In [17]:
# 2010-12-09 is the latest date in the dataset.
df_copy['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

In [18]:
# So let's assume that today is 2010-12-10
today_date = dt.datetime(2010, 12, 10) 
print(today_date) # 2010-12-10

2010-12-10 00:00:00


In [19]:
# Since we want to calculate the rfm metrics in terms of each customer, 
# we will group by CustomerID
rfm = df_copy.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: round(TotalPrice.sum(),2)})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,164,11,372.86
12347.0,2,2,1323.32
12348.0,73,1,222.16
12349.0,42,3,2671.14
12351.0,10,1,300.93


In [20]:
# So we can change the column names..
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,164,11,372.86
12347.0,2,2,1323.32
12348.0,73,1,222.16
12349.0,42,3,2671.14
12351.0,10,1,300.93


In [21]:
# We have 4314 customers right now
print(rfm.shape) # (4314, 3)

(4314, 3)


In [22]:
rfm.describe()

,Recency,Frequency,Monetary
count,4314.000000,4314.000000,4314.000000
mean,90.269819,4.454103,2047.288656
std,96.944304,8.168658,8912.523241
min,0.000000,1.000000,0.000000
25%,17.000000,1.000000,307.950000
50%,52.000000,2.000000,705.550000
75%,135.000000,5.000000,1722.802500
max,373.000000,205.000000,349164.350000


Recency Score

In [23]:
# Since lower recency means higher recency score, 
# our labels will be [5,4,3,2,1]
rfm["Recency_Score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
print(rfm.head())

             Recency  Frequency  Monetary Recency_Score
Customer ID                                            
12346.0          164         11    372.86             2
12347.0            2          2   1323.32             5
12348.0           73          1    222.16             2
12349.0           42          3   2671.14             3
12351.0           10          1    300.93             5


Monetary Score

In [24]:
# Since higher monetary value means higher score, 
# our labels will be [1,2,3,4,5]
rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])
print(rfm.head())

             Recency  Frequency  Monetary Recency_Score Monetary_Score
Customer ID                                                           
12346.0          164         11    372.86             2              2
12347.0            2          2   1323.32             5              4
12348.0           73          1    222.16             2              1
12349.0           42          3   2671.14             3              5
12351.0           10          1    300.93             5              2


Frequency Score

In [25]:
rfm["Frequency_Score"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
print(rfm.head())

             Recency  Frequency  Monetary Recency_Score Monetary_Score  \
Customer ID                                                              
12346.0          164         11    372.86             2              2   
12347.0            2          2   1323.32             5              4   
12348.0           73          1    222.16             2              1   
12349.0           42          3   2671.14             3              5   
12351.0           10          1    300.93             5              2   

            Frequency_Score  
Customer ID                  
12346.0                   5  
12347.0                   2  
12348.0                   1  
12349.0                   3  
12351.0                   1  


RFM Score

In [26]:
rfm_score = pd.DataFrame()

In [27]:
# Finally, we can obtain the RFM scores. we won't use the 'monetary' metric 
# for RFM score, we will only use 'recency' and 'frequency', since 'monetary'
# is not necessary.
rfm_score["RFM_Score"] = (rfm['Recency_Score'].astype(str) 
                    + rfm['Frequency_Score'].astype(str))
rfm_score.head()

,RFM_Score
Customer ID,
12346.0,25
12347.0,52
12348.0,21
12349.0,33
12351.0,51


In [28]:
# For example, let's see some of 'champions' segment
rfm_score[rfm_score["RFM_Score"] == "55"].head()

,RFM_Score
Customer ID,
12415.0,55
12431.0,55
12471.0,55
12472.0,55
12474.0,55


In [29]:

# And see some of 'hibernating' segment
print(rfm_score[rfm_score["RFM_Score"] == "11"].head())


            RFM_Score
Customer ID          
12355.0            11
12362.0            11
12366.0            11
12368.0            11
12378.0            11


Segmentação de Usuário

In [30]:
# Let's create a 'segment' column..
# First, create a segmentation dict by using RegEx
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [31]:
rfm_score['Segment'] = rfm_score['RFM_Score'].replace(seg_map, regex=True)
# And that's all...
rfm_score.head()

,RFM_Score,Segment
Customer ID,,
12346.0,25,cant_loose
12347.0,52,potential_loyalists
12348.0,21,hibernating
12349.0,33,need_attention
12351.0,51,new_customers


In [32]:
# And most of the time, you will need rfm.csv
rfm_score.to_csv("rfm_score_2009_2010.csv")

In [33]:
def create_rfm(df):

    dataframe = df

    # Preparation
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]

    # Calculate RFM metrics
    today_date = dt.datetime(2011, 12, 10)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})

    rfm.columns = ['Recency', 'Frequency', "Monetary"]
    
    # Calculate RFM scores
    rfm["Recency_Score"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["Frequency_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["Monetary_Score"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

    rfm["RFM_Score"] = (rfm['Recency_Score'].astype(str) +
                        rfm['Frequency_Score'].astype(str))


    # Segmentation
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['Segment'] = rfm['RFM_Score'].replace(seg_map, regex=True)
    rfm = rfm[["Recency", "Frequency", "Monetary", "Segment"]]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm


In [34]:
create_rfm(df)

,Recency,Frequency,Monetary,Segment
Customer ID,,,,
12346,529,11,372.86,cant_loose
12347,367,2,1323.32,potential_loyalists
12348,438,1,222.16,hibernating
12349,407,3,2671.14,need_attention
12351,375,1,300.93,new_customers
...,...,...,...,...
18283,382,6,641.77,loyal_customers
18284,431,1,461.68,about_to_sleep
18285,660,1,427.00,hibernating
